In [ ]:
# Train model
import sys
sys.path.append(r'D:\@web_developement\FLASK\skin_disease_detection\backend\app')
from models import cnn_model

In [ ]:
# Import saved model
from keras.models import load_model
model = load_model(r'D:\@web_developement\FLASK\skin_disease_detection\backend\app\static\trained_models\skin_disease_model.h5')
print(model)